In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup as bts
import time
from urllib.parse import urljoin

def getAndParseURL(url):
    driver.get(url)
    time.sleep(2)
    html = driver.page_source
    soup = bts(html, 'html.parser')
    return soup

# Webdriver'ı başlat
driver = webdriver.Chrome()

base_url = 'https://www.sikayetvar.com/turkcell/yurt-disi-paketleri'
# Trendyol sayfalarını dolaşarak ürün linklerini çek
pages = []
for page in range(1, 31):  # Örnek olarak 1'den 30'a kadar olan sayfaları ziyaret ediyoruz
    pages.append("https://www.sikayetvar.com/turkcell/yurt-disi-paketleri?page=" + str(page))

products = []
for page in pages:
    soup = getAndParseURL(page)
    for h2 in soup.find_all('h2', class_='complaint-title'):
        a_tag = h2.find('a')
        if a_tag and 'href' in a_tag.attrs:
            href = a_tag['href']
            full_url = urljoin(base_url, href)
            products.append(full_url)

all_yorumlar = []

def getProductDetails(product_url):
    # Web sayfasını aç
    html = getAndParseURL(product_url)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'h1.complaint-detail-title')))
    time.sleep(2)
    product_name = driver.find_element(By.CSS_SELECTOR, 'h1.complaint-detail-title').text
    comment_paragraphs = driver.find_elements(By.CSS_SELECTOR, 'div.complaint-detail-description')

    comments = [paragraph.text for paragraph in comment_paragraphs]

    # Her bir ürün için bir sözlük oluştur
    product_dict = {
        'yorum': comments,
        'product_name': product_name
    }

    return product_dict

for product in products:
    details = getProductDetails(product)
    all_yorumlar.append(details)

# Tarayıcıyı kapatma
driver.quit()

# Sonuçları yazdırma
for yorum in all_yorumlar:
    print(yorum)


{'yorum': ['Turkcell yurt dışına açık hat ancak arama yapmıyor, mesaj, internet hiçbir şey kullanılamıyor. Çocuğumun hattını yurt dışına çıkmadan önce açtırmama rağmen arama yapılamıyor. Karşıdan arama yapılamıyor. İnternet ve mesaj da kullanılamıyor. Tüm ayarlar yapılmasına rağmen çalışmıyor. Destek talebimize de cevap gelmedi.'], 'product_name': 'Turkcell Hat Yurt Dışına Açılmıyor'}
{'yorum': ["22 Temmuz pazartesi günü 0506 ****46 numaralı telefon hattım için 400 TL'ye yurt dışı gezgin paketi satın aldım ve İtalya'ya gittim. İtalya'dayken hattımla telefon araması ve SMS gönderimi gerçekleştirebildim ama internete bağlanamadım. 23 Temmuz salı günü bu paketin yapılamadığını düşündüğüm için bulduğum ilk Wi-Fi noktasından mobil uygulamaya girip paketin üzerinde görülmemesi üzerine yeniden bir yurt dışı gezgin paketi daha satın aldım. Yarım saat sonra internetin gelmemesi üzerine müşteri hizmetlerini aradım ve ve ikinci defa mobil uygulama üzerinden paket aldığımı ve bunu iptal etmek iste

In [9]:
import pandas as pd
# Create a DataFrame from the collected data
df = pd.DataFrame(all_yorumlar)
df.to_csv('yeni_yurt_dışı.csv', index=False)
df

,yorum,product_name
0,[Turkcell yurt dışına açık hat ancak arama yap...,Turkcell Hat Yurt Dışına Açılmıyor
1,[22 Temmuz pazartesi günü 0506 ****46 numaralı...,Turkcell Hattım Kullanıma Kapalı
2,[Turkcell kullanıcısıyım. 24.07.2024 tarihinde...,Turkcell Yurt Dışı Kullanım Ücreti
3,[Yurt dışına çıkmadan önce yurt dışı paketi al...,Muhatap Bulamıyorum Turkcell Beni Mobile Yönle...
4,[Turkcell yurt dışına mesaj gönderme. Hiçbir ş...,Turkcell Yurt Dışı SMS Sorunu
...,...,...
675,[İPhone 14 Pro Max yurt dışı cihazım var. 07 T...,Turkcell Hattımda Şebeke Sorunu Ve Mağduriyet
676,[İPhone 14 Pro yurt dışı cihazım 07 Temmuz 202...,Turkcell Yurt Dışı Cihazı Şebeke Sorunu
677,[Turkcell firması hatta ek faturalı tarifeden ...,Turkcell Ürün Tesliminin Gecikmesi.
678,[Turkcell yurt dışı çevresinde her bulunduğumd...,Turkcell Yurt Dışı Tarifesi


In [7]:
# List içeren sütunu tuple'a dönüştürme
df['yorum'] = df['yorum'].apply(tuple)
duplicates = df.duplicated()
print("Duplicate kayıtlar var mı:")
duplicates

Duplicate kayıtlar var mı:


0      False
1      False
2      False
3      False
4      False
       ...  
675    False
676    False
677    False
678    False
679    False
Length: 680, dtype: bool

In [8]:
df

,yorum,product_name
0,(Turkcell yurt dışına açık hat ancak arama yap...,Turkcell Hat Yurt Dışına Açılmıyor
1,(22 Temmuz pazartesi günü 0506 ****46 numaralı...,Turkcell Hattım Kullanıma Kapalı
2,(Turkcell kullanıcısıyım. 24.07.2024 tarihinde...,Turkcell Yurt Dışı Kullanım Ücreti
3,(Yurt dışına çıkmadan önce yurt dışı paketi al...,Muhatap Bulamıyorum Turkcell Beni Mobile Yönle...
4,(Turkcell yurt dışına mesaj gönderme. Hiçbir ş...,Turkcell Yurt Dışı SMS Sorunu
...,...,...
675,(İPhone 14 Pro Max yurt dışı cihazım var. 07 T...,Turkcell Hattımda Şebeke Sorunu Ve Mağduriyet
676,(İPhone 14 Pro yurt dışı cihazım 07 Temmuz 202...,Turkcell Yurt Dışı Cihazı Şebeke Sorunu
677,(Turkcell firması hatta ek faturalı tarifeden ...,Turkcell Ürün Tesliminin Gecikmesi.
678,(Turkcell yurt dışı çevresinde her bulunduğumd...,Turkcell Yurt Dışı Tarifesi


In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup as bts
import time
from urllib.parse import urljoin

def getAndParseURL(url):
    driver.get(url)
    time.sleep(2)
    html = driver.page_source
    soup = bts(html, 'html.parser')
    return soup

# Webdriver'ı başlat
driver = webdriver.Chrome()

base_url = 'https://www.sikayetvar.com/turkcell/fatura'
# Trendyol sayfalarını dolaşarak ürün linklerini çek
pages = []
for page in range(39, 60):  # Örnek olarak 1'den 19'a kadar olan sayfaları ziyaret ediyoruz
    pages.append("https://www.sikayetvar.com/turkcell/fatura?page=" + str(page))

products = []
for page in pages:
    soup = getAndParseURL(page)
    for h2 in soup.find_all('h2', class_='complaint-title'):
        a_tag = h2.find('a')
        if a_tag and 'href' in a_tag.attrs:
            href = a_tag['href']
            full_url = urljoin(base_url, href)
            products.append(full_url)

all_yorumlar = []

def getProductDetails(product_url):
    # Web sayfasını aç
    html = getAndParseURL(product_url)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'h1.complaint-detail-title')))
    time.sleep(2)
    product_name = driver.find_element(By.CSS_SELECTOR, 'h1.complaint-detail-title').text
    comment_paragraphs = driver.find_elements(By.CSS_SELECTOR, 'div.complaint-detail-description')

    comments = [paragraph.text for paragraph in comment_paragraphs]

    # Her bir ürün için bir sözlük oluştur
    product_dict = {
        'yorum': comments,
        'product_name': product_name
    }

    return product_dict

for product in products:
    details = getProductDetails(product)
    all_yorumlar.append(details)

# Tarayıcıyı kapatma
driver.quit()

# Sonuçları yazdırma
for yorum in all_yorumlar:
    print(yorum)


WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=126.0.6478.183)
Stacktrace:
	GetHandleVerifier [0x00007FF6A8ECEEB2+31554]
	(No symbol) [0x00007FF6A8E47EE9]
	(No symbol) [0x00007FF6A8D0872A]
	(No symbol) [0x00007FF6A8CEFA9C]
	(No symbol) [0x00007FF6A8D1630D]
	(No symbol) [0x00007FF6A8D9CCD5]
	(No symbol) [0x00007FF6A8D7CDD3]
	(No symbol) [0x00007FF6A8D4A33B]
	(No symbol) [0x00007FF6A8D4AED1]
	GetHandleVerifier [0x00007FF6A91D8B2D+3217341]
	GetHandleVerifier [0x00007FF6A9225AF3+3532675]
	GetHandleVerifier [0x00007FF6A921B0F0+3489152]
	GetHandleVerifier [0x00007FF6A8F7E786+750614]
	(No symbol) [0x00007FF6A8E5376F]
	(No symbol) [0x00007FF6A8E4EB24]
	(No symbol) [0x00007FF6A8E4ECB2]
	(No symbol) [0x00007FF6A8E3E17F]
	BaseThreadInitThunk [0x00007FFB2DAB7374+20]
	RtlUserThreadStart [0x00007FFB2E55CC91+33]


In [2]:
import pandas as pd
# Create a DataFrame from the collected data
df = pd.DataFrame(all_yorumlar)
df.to_csv('yeni_fatura3.csv', index=False)
df

,yorum,product_name
0,[Turkcell Bandırma bayisi 2 ay önce 290 TL kam...,Turkcell Yanıltıcı Kampanya Ve Çözüm Üretmeyen...
1,[Turkcell’den 01.06.2023 tarihinde 12 ay taahh...,Turkcell Haksız Yere Bir Fatura Daha Kesiyor
2,[Sözleşmemin bitmesine birkaç gün kala Turkcel...,Turkcell Abonelik Yenilemesinde Beklenmedik Fa...
3,[Turkcell yıllık kontratımın bitmesine 3 gün v...,Turkcell Haksız Kazanç Elde Etmek
4,[Yasal hak sahibi olmadığım bir hattın faturas...,Turkcell Hat Sahibi Değilim Başkasının Faturas...
...,...,...
131,[Turkcell pasajdan Sinbo'nun SCM2938 modelini ...,Sinbo Filtre Kahve Makinesi Turkcell Pasajdan ...
132,[Turkcell otomatik ödeme talimatım varken fatu...,Turkcell Otomatik Ödeme Talimatının Çalışmaması
133,"[Bana sürekli Turkcell'den, data hattı borcunu...",Turkcell'in Alakam Olmayan Fatura Borç Mesajı ...
134,[Turkcell tarafından kullanmadığım vergi borçl...,Turkcell Kazandığınız Paralar Haksız Yere Mill...
